In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-03 18:59:03--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-03 18:59:04 (23.0 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [12]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [9]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [13]:
query = "Can I still enroll in the course?"

In [11]:
print(llm(query))

 Yes, you can potentially still enroll if:

1. The application window hasn't closed yet and there are more spots available than students who have applied. 
2. Your eligibility has not expired (usually due to a cut-off time for applications). If your deadline is approaching, it would be wise to submit promptly before the closing date or confirm with an enrollment officer that you still qualify despite being past some unspecified threshold in time since application submission.
3. There are no further prerequisites left undone which may disqualify late applicants (e.g., missing documentation, assignment completion). 
4. You have not been previously rejected for enrolling and if there's still a chance to reapply after the deadline passes given special circumstances or exceptions by course administrators such as financial hardships affecting your ability to pay tuition fees immediately upon confirmation of acceptance (in some cases, scholarships can be applied).
5. Lastly, it is always advi

In [14]:
print(rag(query))

 Yes, even if you miss the start date of our course that begins at Google Cloud's public calendar for Computer Science in Data Analytics and Visualization (starting from Jan 15th), 2024 at 17h00 time zone on desktop devices, as a self-paced student who missed registration before the start date, you are still able to submit your homework assignments. However, remember there will be final project deadlines after submission of these tasks and don't procrastinate with this assignment due for late submissions being accepted till Mar 1st only from midnight-2am time zone on GMT+8 timezone as well noted in our FAQ document that captures frequently asked technical questions.
You can continue your learning journey, by following the course materials stored post termination of said program and prepare yourself for participation with subsequent cohorts through completion preparation activities such as continued homework submission or final capstone project work using channels like Google Calendar a